In [ ]:
import pandas as pd
import requests
from datetime import datetime
from time import sleep

# Load your data (this file must already include Latitude, Longitude, Date, and Time)
df = pd.read_csv("Map_Overview_with_Weather.csv")

# Your Visual Crossing API key
API_KEY = "insert Vising Crossing API key here"

# Cache for already-queried (rounded_lat, rounded_lon, date) tuples
wetness_cache = {}

def extract_datetime(row):
    try:
        date_str = row['Collision_Date_Tooltip'].split('@')[0].strip()
        date_obj = datetime.strptime(date_str, "%A, %B %d, %Y")
        date_fmt = date_obj.strftime("%Y-%m-%d")
        time_str = row['Collision_Time_adj'].strip()
        if time_str.lower() == "unknown":
            return date_fmt, None
        return date_fmt, time_str
    except:
        return None, None

def get_road_wetness(lat, lon, date_str, time_str):
    base_url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/"
    location = f"{lat},{lon}"
    url = f"{base_url}{location}/{date_str}?key={API_KEY}&unitGroup=us&include=hours"

    while True:
        try:
            response = requests.get(url)
            if response.status_code == 200:
                break
            elif response.status_code == 429:
                print(" → Hit rate limit (429). Sleeping for 60 seconds...")
                sleep(60)
            else:
                return f"HTTP {response.status_code}"
        except Exception as e:
            return f"Exception: {e}"

    try:
        data = response.json()
        hours = data['days'][0].get('hours', [])
        if not hours:
            return "No hourly data"

        crash_time = datetime.strptime(time_str, "%H:%M")
        crash_hour = round(crash_time.hour + crash_time.minute / 60)

        # Accumulate precipitation over the 6 hours before and including crash time
        start_hour = max(0, crash_hour - 6)
        precip_total = 0.0
        for h in hours[start_hour:crash_hour+1]:
            try:
                precip_total += float(h.get('precip', 0))
            except:
                continue

        # Assign graded wetness level
        if precip_total > 5:
            return "3 - Soaking"
        elif precip_total > 1:
            return "2 - Very Wet"
        elif precip_total > 0:
            return "1 - Damp"
        else:
            return "0 - Dry"
    except Exception as e:
        return f"Exception: {e}"

# Collect road wetness data
wetness_levels = []

for i, row in df.iterrows():
    lat, lon = row['Latitude_adj'], row['Longitude_adj']
    date_str, time_str = extract_datetime(row)
    cache_key = (date_str, round(lat, 2), round(lon, 2))

    print(f"[{i+1}/{len(df)}] Getting road wetness for {date_str} {time_str or '[unknown]'} at ({lat},{lon})")

    if date_str and time_str:
        if cache_key in wetness_cache:
            wetness = wetness_cache[cache_key]
            print(" → (cached)")
        else:
            wetness = get_road_wetness(lat, lon, date_str, time_str)
            wetness_cache[cache_key] = wetness
    else:
        wetness = "Missing or invalid time/date"

    print(f" → Wetness: {wetness}")
    wetness_levels.append(wetness)

    # Save partial results every 50 rows
    if i > 0 and i % 50 == 0:
        temp_df = df.iloc[:i+1].copy()
        temp_df["Road_Wetness_Level"] = wetness_levels
        temp_df.to_csv("Map_Overview_with_Wetness_partial.csv", index=False)
        print("Saved checkpoint: Map_Overview_with_Wetness_partial.csv")

    sleep(2)

# Final save
df["Road_Wetness_Level"] = wetness_levels
df.to_csv("Map_Overview_with_Weather_and_Wetness.csv", index=False)
print("✅ Done! File saved as Map_Overview_with_Weather_and_Wetness.csv")
